In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 필요한 라이브러리 import
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
# 데이터 불러오기
file_path = '/content/drive/MyDrive/result_meal.csv'  # Colab에 파일을 업로드하고 이 경로를 확인하세요
df = pd.read_csv(file_path)

# 'text_review' 컬럼이 있는지 확인
if 'text_review' not in df.columns:
    print("Error: 'text_review' 컬럼이 데이터프레임에 없습니다.")
else:
    # 사용자 텍스트 입력 받기
    user_input = input("비교할 텍스트를 입력하세요: ")

    # KOBERT 모델과 토크나이저 로드
    tokenizer = BertTokenizer.from_pretrained('skt/kobert-base-v1')
    model = BertModel.from_pretrained('skt/kobert-base-v1')

    # 텍스트를 임베딩하는 함수 정의
    def embed_text(text):
        tokens = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)
        with torch.no_grad():
            output = model(**tokens)
        # 일반적으로 BERT의 마지막 레이어의 CLS 토큰 벡터를 임베딩으로 사용
        embedding = output.last_hidden_state[:, 0, :].numpy()
        return embedding

    # 입력 텍스트와 데이터프레임의 리뷰 텍스트들을 임베딩
    user_embedding = embed_text(user_input)

    reviews_embedding = [embed_text(review) for review in df['text_review'].tolist()]

    # 코사인 유사도 계산
    similarity_scores = [cosine_similarity(user_embedding, review_emb)[0][0] for review_emb in reviews_embedding]

    # 유사도 점수를 데이터프레임에 추가
    df['similarity_score'] = similarity_scores

    # 유사도 점수가 높은 순으로 정렬
    df_sorted = df.sort_values(by='similarity_score', ascending=False)

    # 상위 5개 결과 출력
    print("입력 텍스트와 유사한 리뷰 상위 4개:")
    print(df_sorted[['store_name', 'text_review', 'similarity_score']].head(4))

    # 정렬된 결과를 CSV로 저장 (원하는 경우)
    df_sorted.to_csv('/content/sorted_similarity_scores.csv', index=False)
    print("유사도 점수가 포함된 결과가 sorted_similarity_scores.csv 파일로 저장되었습니다.")


비교할 텍스트를 입력하세요: 조용하고 아늑한 분위기


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/244 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'BertTokenizer'.


TypeError: stat: path should be string, bytes, os.PathLike or integer, not NoneType

In [8]:

# 필요한 라이브러리 import
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# 데이터 불러오기
file_path = '/content/drive/MyDrive/result_meal.csv'  # Colab에 파일을 업로드하고 이 경로를 확인하세요
df = pd.read_csv(file_path)

In [11]:
import re

# 이모티콘과 특수 문자를 제거하는 함수
def clean_text(text):
    # 입력값이 NaN이면 그대로 반환
    if pd.isna(text):
        return text

    # 유니코드 이모티콘 제거
    text = re.sub(r'[\U00010000-\U0010ffff]', '', text)
    # 특수 문자와 공백을 제거하고 문장 부호를 최소화
    text = re.sub(r'[^\w\s]', ' ', text)
    # 연속된 공백을 하나의 공백으로 압축
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# 텍스트 정제
df['cleaned_text'] = df['text_review'].apply(clean_text)

In [12]:
# 'text_review' 컬럼이 있는지 확인
if 'text_review' not in df.columns:
    print("Error: 'text_review' 컬럼이 데이터프레임에 없습니다.")
else:
    # NaN 값을 제거하거나 빈 문자열로 대체
    df['cleaned_text'] = df['cleaned_text'].fillna('')

    # 사용자 텍스트 입력 받기
    user_input = input("비교할 텍스트를 입력하세요: ")

    # 모델 로드 (SBERT 사용)
    model = SentenceTransformer('all-MiniLM-L6-v2')

    # 입력 텍스트와 데이터프레임의 리뷰 텍스트들을 임베딩
    user_embedding = model.encode([user_input])
    reviews_embedding = model.encode(df['cleaned_text'].tolist())

    # 코사인 유사도 계산
    similarity_scores = cosine_similarity(user_embedding, reviews_embedding)[0]

    # 유사도 점수를 데이터프레임에 추가
    df['similarity_score'] = similarity_scores

    # 유사도 점수가 높은 순으로 정렬
    df_sorted = df.sort_values(by='similarity_score', ascending=False)

    # 상위 5개 결과 출력
    print("입력 텍스트와 유사한 리뷰 상위 5개:")
    print(df_sorted[['store_name', 'text_review', 'similarity_score']].head())

    # 정렬된 결과를 CSV로 저장 (원하는 경우)
    df_sorted.to_csv('/content/sorted_similarity_scores.csv', index=False)
    print("유사도 점수가 포함된 결과가 sorted_similarity_scores.csv 파일로 저장되었습니다.")


비교할 텍스트를 입력하세요: 해장하기 좋은 국물
입력 텍스트와 유사한 리뷰 상위 5개:
     store_name                        text_review  similarity_score
2      육쌈냉면 숙대점                   냉면이랑 고기 둘다 맛있어요!          0.837308
587   타코비 숙명여대점        사장님 너무 친절하시고 타코야끼 너무 맛있어요:)          0.836435
1038   남영동양문 본점  음식 다 맛있고 친절하시고 좋아요! 곧 또 방문 예정입니다🤞          0.833320
1328       카페시바            항상 궁금했는데, 너무너무 맛있었어요 ☺️          0.828620
539   홍곱창 숙명여대점                    소금구이 막창 너무 맛있어요          0.826579
유사도 점수가 포함된 결과가 sorted_similarity_scores.csv 파일로 저장되었습니다.


In [13]:
    # 유사도 점수가 높은 순으로 정렬
    df_sorted = df.sort_values(by='similarity_score', ascending=False)

    # store_name별로 유사도 점수 상위 5개의 평균을 계산
    top_5_avg_similarity = (
        df_sorted.groupby('store_name')
        .apply(lambda x: x.nlargest(5, 'similarity_score')['similarity_score'].mean())
        .reset_index(name='top_5_avg_similarity')
    )

    # 평균 유사도 점수가 가장 높은 5개의 store_name 출력
    top_5_avg_similarity_sorted = top_5_avg_similarity.sort_values(by='top_5_avg_similarity', ascending=False).head(5)

    print("유사도 점수 상위 5개의 평균이 가장 높은 상위 5개의 가게:")
    print(top_5_avg_similarity_sorted)

유사도 점수 상위 5개의 평균이 가장 높은 상위 5개의 가게:
     store_name  top_5_avg_similarity
35   빨봉분식 숙명여대점              0.790326
48    아리랑노점 숙대점              0.789264
26  미스터카츠 숙대입구점              0.779017
66    타코비 숙명여대점              0.778816
38         살팀보카              0.777568


<ipython-input-13-42b4cb43be4b>:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.nlargest(5, 'similarity_score')['similarity_score'].mean())


In [ ]:
# 필요한 라이브러리 import
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# 'text_review' 컬럼이 있는지 확인
if 'text_review' not in df.columns:
    print("Error: 'text_review' 컬럼이 데이터프레임에 없습니다.")
else:
    # 사용자 텍스트 입력 받기
    user_input = input("비교할 텍스트를 입력하세요: ")

    # 키워드와 리뷰를 하나의 텍스트로 결합
    def combine_text(row):
        keywords = ' '.join([str(row[col]) for col in ['visit_keyword_1', 'visit_keyword_2', 'visit_keyword_3', 'visit_keyword_4',
                                                       'review_keyword_1', 'review_keyword_2', 'review_keyword_3', 'review_keyword_4', 'review_keyword_5'] if col in row])
        return f"{keywords} {row['text_review']}"

    # 새로운 컬럼으로 키워드와 리뷰 결합 텍스트 생성
    df['combined_text'] = df.apply(combine_text, axis=1)

    # 모델 로드 (SBERT 사용)
    model = SentenceTransformer('all-MiniLM-L6-v2')

    # 입력 텍스트와 데이터프레임의 결합 텍스트들을 임베딩
    user_embedding = model.encode([user_input])
    combined_embeddings = model.encode(df['combined_text'].tolist())

    # 코사인 유사도 계산
    similarity_scores = cosine_similarity(user_embedding, combined_embeddings)[0]

    # 유사도 점수를 데이터프레임에 추가
    df['similarity_score'] = similarity_scores

    # 유사도 점수가 높은 순으로 정렬
    df_sorted = df.sort_values(by='similarity_score', ascending=False)

    # 상위 5개 결과 출력
    print("입력 텍스트와 유사한 리뷰 상위 5개:")
    print(df_sorted[['store_name', 'combined_text', 'similarity_score']].head())

    # 정렬된 결과를 CSV로 저장 (원하는 경우)
    df_sorted.to_csv('/content/sorted_similarity_scores.csv', index=False)
    print("유사도 점수가 포함된 결과가 sorted_similarity_scores.csv 파일로 저장되었습니다.")


비교할 텍스트를 입력하세요: 공부하기 좋아요


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


입력 텍스트와 유사한 리뷰 상위 5개:
      store_name            combined_text  similarity_score
22   벤티프레소 숙명여대점                 가성비 최고예요          0.884818
243    미드스트오브플로우    독특한 느낌조용한 분위기2층은 사용불가          0.848002
35   벤티프레소 숙명여대점            맛있어요 친구랑 자주가용          0.822986
381         을의커피               카공하기 좋아요!!          0.817015
118       카페 퓨엔테   디저트도 맛있고 좋아여자주와서 놀고갑니당          0.788587
유사도 점수가 포함된 결과가 sorted_similarity_scores.csv 파일로 저장되었습니다.
